# Database Setup for Master Birder Paper

This notebook sets up a SQLite database for the master birder paper project.
The database will be stored as a single file in the `data/` directory.

## Prerequisites

Make sure you have all dependencies installed:
```bash
pdm install
```

If you need to add new dependencies later, use:
```bash
pdm add <package-name>
```

For development dependencies:
```bash
pdm add -G dev <package-name>
```


In [3]:
import sqlite3
import os
import pandas as pd
from pathlib import Path

# Set up paths
project_root = Path.cwd().parent  # Go up one level from notebooks/
data_dir = project_root / "data"
db_path = data_dir / "master_birder.db"

print(f"Project root: {project_root}")
print(f"Data directory: {data_dir}")
print(f"Database path: {db_path}")

# Ensure data directory exists
data_dir.mkdir(exist_ok=True)
print(f"Data directory exists: {data_dir.exists()}")


Project root: /Users/ken/Documents/wk/master-birder-paper
Data directory: /Users/ken/Documents/wk/master-birder-paper/data
Database path: /Users/ken/Documents/wk/master-birder-paper/data/master_birder.db
Data directory exists: True


## Database Connection and Setup

Create a connection to the SQLite database. If the database doesn't exist, it will be created automatically.


In [4]:
# Create database connection
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"Connected to database: {db_path}")
print(f"Database file exists: {db_path.exists()}")

# Enable foreign key constraints
cursor.execute("PRAGMA foreign_keys = ON")
print("Foreign key constraints enabled")


Connected to database: /Users/ken/Documents/wk/master-birder-paper/data/master_birder.db
Database file exists: True
Foreign key constraints enabled


## Database Schema Creation

This section creates the Avibase database schema based on the Mermaid diagram in `data/avibase.schema.mer`.

### Avibase Schema

The database includes the following tables:
- **AvibaseID**: Main table with Avibase IDs and concept labels
- **ParentChildRelationships**: Hierarchical relationships between concepts
- **OriginalConcepts**: Links Avibase IDs to original concept IDs
- **TaxanomicConcepts**: Taxonomic information including scientific and common names
- **NameConcepts**: Nomenclatural information (protonyms, authors, publications)
- **LifeHistory**: Life history traits and characteristics
- **GeoGraphicRange**: Geographic distribution information
- **OtherRelationships**: Various relationship types between concepts
- **Synonyms**: Multilingual synonym information


In [5]:
# Avibase schema creation
# Based on the Mermaid diagram in data/avibase.schema.mer

create_tables_sql = """
-- Avibase Database Schema
-- Based on the Mermaid diagram in data/avibase.schema.mer

-- Main Avibase ID table
CREATE TABLE IF NOT EXISTS AvibaseID (
    avibase_id TEXT PRIMARY KEY,
    concept_label TEXT
);

-- Parent-Child relationships table
CREATE TABLE IF NOT EXISTS ParentChildRelationships (
    avibase_id TEXT,
    version TEXT,
    parent_id TEXT,
    fract_weight REAL,
    PRIMARY KEY (avibase_id, version),
    FOREIGN KEY (avibase_id) REFERENCES AvibaseID (avibase_id),
    FOREIGN KEY (parent_id) REFERENCES AvibaseID (avibase_id)
);

-- Original concepts linking table
CREATE TABLE IF NOT EXISTS OriginalConcepts (
    avibase_id TEXT,
    concept_id TEXT,
    PRIMARY KEY (avibase_id, concept_id),
    FOREIGN KEY (avibase_id) REFERENCES AvibaseID (avibase_id)
);

-- Taxonomic concepts table
CREATE TABLE IF NOT EXISTS TaxanomicConcepts (
    concept_id TEXT PRIMARY KEY,
    avibase_id TEXT,
    taxon_name_id TEXT,
    authority TEXT,
    scientific_name TEXT,
    common_name TEXT,
    higher_classification TEXT,
    FOREIGN KEY (avibase_id) REFERENCES AvibaseID (avibase_id),
    FOREIGN KEY (taxon_name_id) REFERENCES NameConcepts (taxon_name_id)
);

-- Name concepts table
CREATE TABLE IF NOT EXISTS NameConcepts (
    taxon_name_id TEXT PRIMARY KEY,
    protonym TEXT,
    authors TEXT,  -- Storing as TEXT since SQLite doesn't have native array support
    year INTEGER,
    publication_source TEXT,
    tsn TEXT
);

-- Life history traits table
CREATE TABLE IF NOT EXISTS LifeHistory (
    avibase_id TEXT,
    trait TEXT,
    reference TEXT,
    value TEXT,
    PRIMARY KEY (avibase_id, trait, reference),
    FOREIGN KEY (avibase_id) REFERENCES AvibaseID (avibase_id)
);

-- Geographic range table
CREATE TABLE IF NOT EXISTS GeoGraphicRange (
    avibase_id TEXT,
    region TEXT,
    status TEXT,  -- Using TEXT instead of ENUM for SQLite compatibility
    PRIMARY KEY (avibase_id, region),
    FOREIGN KEY (avibase_id) REFERENCES AvibaseID (avibase_id)
);

-- Other relationships table
CREATE TABLE IF NOT EXISTS OtherRelationships (
    avibase_id TEXT,
    related_id TEXT,
    relationship_type TEXT,  -- Using TEXT instead of ENUM for SQLite compatibility
    PRIMARY KEY (avibase_id, related_id, relationship_type),
    FOREIGN KEY (avibase_id) REFERENCES AvibaseID (avibase_id),
    FOREIGN KEY (related_id) REFERENCES AvibaseID (avibase_id)
);

-- Synonyms table
CREATE TABLE IF NOT EXISTS Synonyms (
    avibase_id TEXT,
    language TEXT,
    synonym TEXT,
    reference TEXT,
    PRIMARY KEY (avibase_id, language, synonym),
    FOREIGN KEY (avibase_id) REFERENCES AvibaseID (avibase_id)
);

-- Create indexes for better performance
CREATE INDEX IF NOT EXISTS idx_taxonomic_concepts_avibase_id ON TaxanomicConcepts (avibase_id);
CREATE INDEX IF NOT EXISTS idx_taxonomic_concepts_taxon_name_id ON TaxanomicConcepts (taxon_name_id);
CREATE INDEX IF NOT EXISTS idx_parent_child_parent_id ON ParentChildRelationships (parent_id);
CREATE INDEX IF NOT EXISTS idx_life_history_avibase_id ON LifeHistory (avibase_id);
CREATE INDEX IF NOT EXISTS idx_geographic_range_avibase_id ON GeoGraphicRange (avibase_id);
CREATE INDEX IF NOT EXISTS idx_other_relationships_avibase_id ON OtherRelationships (avibase_id);
CREATE INDEX IF NOT EXISTS idx_other_relationships_related_id ON OtherRelationships (related_id);
CREATE INDEX IF NOT EXISTS idx_synonyms_avibase_id ON Synonyms (avibase_id);
"""

# Execute the schema creation
cursor.executescript(create_tables_sql)
conn.commit()

print("Avibase database schema created successfully!")
print("Tables created:")

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
tables = cursor.fetchall()
for table in tables:
    print(f"  - {table[0]}")

print(f"\nTotal tables created: {len(tables)}")


Avibase database schema created successfully!
Tables created:
  - AvibaseID
  - ParentChildRelationships
  - OriginalConcepts
  - TaxanomicConcepts
  - NameConcepts
  - LifeHistory
  - GeoGraphicRange
  - OtherRelationships
  - Synonyms

Total tables created: 9


## Database Verification

Verify that the database was created correctly and show the table structures.


In [10]:
# Show table schemas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    print(f"\nTable: {table_name}")
    print("-" * 50)
    
    # Get table schema
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    for col in columns:
        col_id, name, data_type, not_null, default_val, pk = col
        pk_str = " (PRIMARY KEY)" if pk else ""
        not_null_str = " NOT NULL" if not_null else ""
        default_str = f" DEFAULT {default_val}" if default_val else ""
        print(f"  {name}: {data_type}{not_null_str}{default_str}{pk_str}")

# Show database file size
if db_path.exists():
    file_size = db_path.stat().st_size
    print(f"\nDatabase file size: {file_size:,} bytes ({file_size/1024:.1f} KB)")



Table: AvibaseID
--------------------------------------------------
  avibase_id: TEXT (PRIMARY KEY)
  concept_label: TEXT

Table: ParentChildRelationships
--------------------------------------------------
  avibase_id: TEXT (PRIMARY KEY)
  version: TEXT (PRIMARY KEY)
  parent_id: TEXT
  fract_weight: REAL

Table: OriginalConcepts
--------------------------------------------------
  avibase_id: TEXT (PRIMARY KEY)
  concept_id: TEXT (PRIMARY KEY)

Table: TaxanomicConcepts
--------------------------------------------------
  concept_id: TEXT (PRIMARY KEY)
  avibase_id: TEXT
  taxon_name_id: TEXT
  authority: TEXT
  scientific_name: TEXT
  common_name: TEXT
  higher_classification: TEXT

Table: NameConcepts
--------------------------------------------------
  taxon_name_id: TEXT (PRIMARY KEY)
  protonym: TEXT
  authors: TEXT
  year: INTEGER
  publication_source: TEXT
  tsn: TEXT

Table: LifeHistory
--------------------------------------------------
  avibase_id: TEXT (PRIMARY KEY)
  tra

## Sample Data Insertion (Optional)

Insert some sample data to test the database structure.


In [8]:
# Insert sample Avibase data
# Sample Avibase IDs
sample_avibase_ids = [
    ('AVIBASE-123456', 'American Robin'),
    ('AVIBASE-123457', 'Northern Cardinal'),
    ('AVIBASE-123458', 'Blue Jay'),
    ('AVIBASE-123459', 'House Sparrow'),
    ('AVIBASE-123460', 'House Finch')
]

cursor.executemany(
    "INSERT OR IGNORE INTO AvibaseID (avibase_id, concept_label) VALUES (?, ?)",
    sample_avibase_ids
)
conn.commit()

print(f"Inserted {len(sample_avibase_ids)} Avibase ID records")

# Sample taxonomic concepts
sample_taxonomic_concepts = [
    ('CONCEPT-001', 'AVIBASE-123456', 'TAXON-001', 'Linnaeus', 'Turdus migratorius', 'American Robin', 'Animalia > Chordata > Aves > Passeriformes > Turdidae'),
    ('CONCEPT-002', 'AVIBASE-123457', 'TAXON-002', 'Linnaeus', 'Cardinalis cardinalis', 'Northern Cardinal', 'Animalia > Chordata > Aves > Passeriformes > Cardinalidae'),
    ('CONCEPT-003', 'AVIBASE-123458', 'TAXON-003', 'Linnaeus', 'Cyanocitta cristata', 'Blue Jay', 'Animalia > Chordata > Aves > Passeriformes > Corvidae'),
    ('CONCEPT-004', 'AVIBASE-123459', 'TAXON-004', 'Linnaeus', 'Passer domesticus', 'House Sparrow', 'Animalia > Chordata > Aves > Passeriformes > Passeridae'),
    ('CONCEPT-005', 'AVIBASE-123460', 'TAXON-005', 'Müller', 'Haemorhous mexicanus', 'House Finch', 'Animalia > Chordata > Aves > Passeriformes > Fringillidae')
]

cursor.executemany(
    "INSERT OR IGNORE INTO TaxanomicConcepts (concept_id, avibase_id, taxon_name_id, authority, scientific_name, common_name, higher_classification) VALUES (?, ?, ?, ?, ?, ?, ?)",
    sample_taxonomic_concepts
)
conn.commit()

print(f"Inserted {len(sample_taxonomic_concepts)} taxonomic concept records")

# Sample name concepts
sample_name_concepts = [
    ('TAXON-001', 'Turdus migratorius', 'Linnaeus', 1758, 'Systema Naturae', 'TSN-179759'),
    ('TAXON-002', 'Cardinalis cardinalis', 'Linnaeus', 1758, 'Systema Naturae', 'TSN-179124'),
    ('TAXON-003', 'Cyanocitta cristata', 'Linnaeus', 1758, 'Systema Naturae', 'TSN-179681'),
    ('TAXON-004', 'Passer domesticus', 'Linnaeus', 1758, 'Systema Naturae', 'TSN-179620'),
    ('TAXON-005', 'Haemorhous mexicanus', 'Müller', 1776, 'Des Ritters Carl von Linné vollständiges Natursystem', 'TSN-179250')
]

cursor.executemany(
    "INSERT OR IGNORE INTO NameConcepts (taxon_name_id, protonym, authors, year, publication_source, tsn) VALUES (?, ?, ?, ?, ?, ?)",
    sample_name_concepts
)
conn.commit()

print(f"Inserted {len(sample_name_concepts)} name concept records")

# Sample life history data
sample_life_history = [
    ('AVIBASE-123456', 'Habitat', 'REF-001', 'Forests, gardens, parks'),
    ('AVIBASE-123456', 'Diet', 'REF-001', 'Omnivorous - insects, fruits, berries'),
    ('AVIBASE-123457', 'Habitat', 'REF-002', 'Woodlands, gardens, shrublands'),
    ('AVIBASE-123457', 'Diet', 'REF-002', 'Seeds, fruits, insects'),
    ('AVIBASE-123458', 'Habitat', 'REF-003', 'Forests, parks, suburban areas'),
    ('AVIBASE-123458', 'Diet', 'REF-003', 'Omnivorous - nuts, seeds, insects')
]

cursor.executemany(
    "INSERT OR IGNORE INTO LifeHistory (avibase_id, trait, reference, value) VALUES (?, ?, ?, ?)",
    sample_life_history
)
conn.commit()

print(f"Inserted {len(sample_life_history)} life history records")


Inserted 5 Avibase ID records


IntegrityError: FOREIGN KEY constraint failed

## Data Verification

Verify that the data was inserted correctly.


In [11]:
# Query Avibase data
print("Avibase IDs in database:")
print("=" * 50)
avibase_df = pd.read_sql_query("SELECT * FROM AvibaseID", conn)
print(avibase_df)

print("\nTaxonomic Concepts in database:")
print("=" * 50)
taxonomic_df = pd.read_sql_query("""
    SELECT tc.*, nc.protonym, nc.authors, nc.year, nc.publication_source
    FROM TaxanomicConcepts tc 
    JOIN NameConcepts nc ON tc.taxon_name_id = nc.taxon_name_id
    ORDER BY tc.scientific_name
""", conn)
print(taxonomic_df)

print("\nLife History data in database:")
print("=" * 50)
life_history_df = pd.read_sql_query("""
    SELECT lh.*, ai.concept_label
    FROM LifeHistory lh 
    JOIN AvibaseID ai ON lh.avibase_id = ai.avibase_id
    ORDER BY lh.avibase_id, lh.trait
""", conn)
print(life_history_df)


Avibase IDs in database:
       avibase_id      concept_label
0  AVIBASE-123456     American Robin
1  AVIBASE-123457  Northern Cardinal
2  AVIBASE-123458           Blue Jay
3  AVIBASE-123459      House Sparrow
4  AVIBASE-123460        House Finch

Taxonomic Concepts in database:
Empty DataFrame
Columns: [concept_id, avibase_id, taxon_name_id, authority, scientific_name, common_name, higher_classification, protonym, authors, year, publication_source]
Index: []

Life History data in database:
Empty DataFrame
Columns: [avibase_id, trait, reference, value, concept_label]
Index: []


## Clean Up Sample Data (Optional)

Run this cell to remove all sample data from the database. This is useful when you want to start fresh with real data or clean up after testing.


In [15]:
# Clean up all sample data
# WARNING: This will delete all data from all tables!

# Get a fresh connection
conn = get_db_connection()
cursor = conn.cursor()

# Count records before deletion
print("Records before cleanup:")
tables_to_clean = [
    'Synonyms', 'OtherRelationships', 'GeoGraphicRange', 'LifeHistory',
    'NameConcepts', 'TaxanomicConcepts', 'OriginalConcepts', 
    'ParentChildRelationships', 'AvibaseID'
]

for table in tables_to_clean:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"  {table}: {count} records")

print("\nCleaning up sample data...")

# Delete in reverse order to respect foreign key constraints
delete_order = [
    'Synonyms', 'OtherRelationships', 'GeoGraphicRange', 'LifeHistory',
    'NameConcepts', 'TaxanomicConcepts', 'OriginalConcepts', 
    'ParentChildRelationships', 'AvibaseID'
]

for table in delete_order:
    cursor.execute(f"DELETE FROM {table}")
    deleted_count = cursor.rowcount
    print(f"Deleted {deleted_count} records from {table}")

conn.commit()

print("\nRecords after cleanup:")
for table in tables_to_clean:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    count = cursor.fetchone()[0]
    print(f"  {table}: {count} records")

print("\n✅ Sample data cleanup completed!")
print("Database is now empty and ready for real data.")

# Close the connection
conn.close()


Records before cleanup:
  Synonyms: 0 records
  OtherRelationships: 0 records
  GeoGraphicRange: 0 records
  LifeHistory: 0 records
  NameConcepts: 0 records
  TaxanomicConcepts: 0 records
  OriginalConcepts: 0 records
  ParentChildRelationships: 0 records
  AvibaseID: 5 records

Cleaning up sample data...


OperationalError: database is locked

## Database Management Functions

Helper functions for common database operations.


In [16]:
def get_db_connection():
    """Get a connection to the database."""
    return sqlite3.connect(db_path)

def execute_query(query, params=None):
    """Execute a query and return results as a DataFrame."""
    conn = get_db_connection()
    try:
        if params:
            df = pd.read_sql_query(query, conn, params=params)
        else:
            df = pd.read_sql_query(query, conn)
        return df
    finally:
        conn.close()

def execute_update(query, params=None):
    """Execute an update/insert/delete query."""
    conn = get_db_connection()
    try:
        cursor = conn.cursor()
        if params:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
        conn.commit()
        return cursor.rowcount
    finally:
        conn.close()

print("Database helper functions defined:")
print("- get_db_connection(): Get a database connection")
print("- execute_query(query, params): Execute SELECT queries")
print("- execute_update(query, params): Execute INSERT/UPDATE/DELETE queries")


Database helper functions defined:
- get_db_connection(): Get a database connection
- execute_query(query, params): Execute SELECT queries
- execute_update(query, params): Execute INSERT/UPDATE/DELETE queries


## Cleanup

Close the database connection when done.


In [17]:
# Close the connection
conn.close()
print("Database connection closed.")
print(f"Database file saved at: {db_path}")
print(f"File size: {db_path.stat().st_size:,} bytes")


Database connection closed.
Database file saved at: /Users/ken/Documents/wk/master-birder-paper/data/master_birder.db
File size: 118,784 bytes


## Next Steps

1. **Provide Schema**: When you're ready, provide the Mermaid diagram of your database schema
2. **Update Schema**: Replace the placeholder schema in this notebook with your actual schema
3. **Data Migration**: If you have existing data, create migration scripts
4. **Integration**: Use the helper functions in your analysis notebooks

## Dependencies

Current dependencies are sufficient for SQLite database operations:
- `sqlite3` (built into Python)
- `pandas` (for data manipulation)
- `pathlib` (for path handling)

No additional PDM dependencies needed for basic SQLite functionality.
